In [39]:
import pandas as pd
import numpy as np
import zipfile
import io
import requests
import os

In [40]:
# Create data folder
os.makedirs("data", exist_ok=True)

# Paths
local_zip_path = "data/18100004-eng.zip"
csv_path = "data/18100004-eng.csv"

# Download zip if not already on disk
if not os.path.exists(local_zip_path):
    print("Downloading zip file...")
    url = "https://www150.statcan.gc.ca/n1/tbl/csv/18100004-eng.zip"
    r = requests.get(url)
    with open(local_zip_path, "wb") as f:
        f.write(r.content)
    print(f"Saved zip to {os.path.abspath(local_zip_path)}")
else:
    print(f"Zip file already exists at {os.path.abspath(local_zip_path)}")

# Extract CSV if not already extracted
if not os.path.exists(csv_path):
    print("Extracting CSV from zip...")
    with zipfile.ZipFile(local_zip_path, "r") as z:
        # Assume first file inside zip is the CSV
        filename = z.namelist()[0]
        z.extract(filename, "data")
        # Rename to consistent csv_path
        os.rename(f"data/{filename}", csv_path)
    print(f"CSV saved to {os.path.abspath(csv_path)}")
else:
    print(f"CSV already exists at {os.path.abspath(csv_path)}")


Zip file already exists at d:\Code\projects\CPI\data\18100004-eng.zip
CSV already exists at d:\Code\projects\CPI\data\18100004-eng.csv


In [41]:
df = pd.read_csv(csv_path)
print("DataFrame loaded successfully")

C:\Users\qdang\AppData\Local\Temp\ipykernel_9064\285962557.py:1: DtypeWarning: Columns (11,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_path)


DataFrame loaded successfully


In [ ]:
# replace placeholder symbols with NaN
df.replace(['..', 'NaN', 'n/a', '', ' '], np.nan, inplace=True)

# convert REF_DATE to datetime
df['REF_DATE'] = pd.to_datetime(df['REF_DATE'], errors='coerce')

# convert VALUE to numeric
df['VALUE'] = pd.to_numeric(df['VALUE'], errors='coerce')


In [43]:
print(df.shape)

(1128853, 15)


In [52]:
import hashlib
def encode_col(name: str) -> str:
    hash_suffix = hashlib.md5(name.encode()).hexdigest()[:8]
    safe_name = ''.join(c if c.isalnum() else '_' for c in name)
    return (safe_name[:40] + '_' + hash_suffix).lower()

In [60]:
# Step 1: Encode product names
df_long = df.copy()
df_long['Encoded_Product'] = df_long['Products and product groups'].apply(encode_col)

# Step 2: Sort for pct_change calculations
df_long = df_long.sort_values(['GEO', 'Encoded_Product', 'REF_DATE'])

# Step 3: Compute MoM and YoY per product+geo
df_long['MoM'] = df_long.groupby(['GEO', 'Encoded_Product'])['VALUE'].pct_change() * 100
df_long['YoY'] = df_long.groupby(['GEO', 'Encoded_Product'])['VALUE'].pct_change(12) * 100

# Optional: forward/backward fill missing
df_long.fillna(method='ffill', inplace=True)
df_long.fillna(method='bfill', inplace=True)

In [61]:
print(df_long.shape)

(1128853, 18)


In [62]:
# Step 4: Keep only the numeric columns for melting
id_cols = ['REF_DATE', 'GEO', 'UOM', 'Encoded_Product', 'Products and product groups']
value_cols = ['VALUE', 'MoM', 'YoY']

df_long_tidy = df_long.melt(
    id_vars=id_cols,
    value_vars=value_cols,
    var_name="Metric",
    value_name="Value"
)

# Step 5: Final column order
df_long_tidy = df_long_tidy[['REF_DATE', 'GEO', 'UOM', 'Products and product groups', 'Encoded_Product', 'Metric', 'Value']]


In [63]:
print(df_long.shape)

(1128853, 18)


In [64]:
print(df_long.head())

        REF_DATE      GEO        DGUID Products and product groups       UOM  \
40772 1978-09-01  Alberta  2016A000248         Alcoholic beverages  2002=100   
42094 1978-10-01  Alberta  2016A000248         Alcoholic beverages  2002=100   
43420 1978-11-01  Alberta  2016A000248         Alcoholic beverages  2002=100   
44746 1978-12-01  Alberta  2016A000248         Alcoholic beverages  2002=100   
46072 1979-01-01  Alberta  2016A000248         Alcoholic beverages  2002=100   

       UOM_ID SCALAR_FACTOR  SCALAR_ID     VECTOR  COORDINATE  VALUE STATUS  \
40772      17         units          0  v41692435      23.257   25.9      E   
42094      17         units          0  v41692435      23.257   25.9      E   
43420      17         units          0  v41692435      23.257   26.0      E   
44746      17         units          0  v41692435      23.257   26.0      E   
46072      17         units          0  v41692435      23.257   26.0      E   

       SYMBOL TERMINATED  DECIMALS          

In [65]:
from sqlalchemy import create_engine, text

user = "postgres"
host = "db.rtewftvldajjhqjbwwfx.supabase.co"
port = "5432"
database = "postgres"
password_path = './password'
table_name = "cpi_long"

with open(password_path, 'r') as f:
    password = f.readline()

db_url = f"postgresql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(db_url, pool_pre_ping=True)
# Keep long format
df_long.to_sql('cpi_long', engine, if_exists='replace', index=False)
print("Loaded CPI long data into Postgres!")

Loaded CPI long data into Postgres!
